In [1]:
import pickle
import pandas as pd
import numpy as np
import gc; gc.enable()
from sklearn.decomposition import TruncatedSVD
from scipy.sparse import hstack, load_npz

In [2]:
'''
with open('all_features.pickle', 'rb') as handle:
    all_features = pickle.load(handle)
'''      
all_features = load_npz('all_features_cv_charwb24.npz')

In [3]:
train_y = pd.read_csv("regression_target.csv")
train_y = train_y['deal_probability'].values
train_len = train_y.shape[0]
print(train_len)

1503424


In [4]:
train_features = all_features[:train_len, :]
del all_features; gc.collect()

0

In [5]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import KFold, GridSearchCV, ParameterGrid, train_test_split
from sklearn.metrics import mean_squared_error, make_scorer
from copy import deepcopy as cp
from wordbatch.models import FM_FTRL

In [6]:
def clip_rmse(ground_truth, predictions):
    predictions = np.clip(predictions, 0., 1.)
    return mean_squared_error(ground_truth, predictions)**.5

clip_rmse_scorer = make_scorer(clip_rmse, greater_is_better=False)

In [7]:
def simple_train_val_split_eval(default_params, X, y, params, partial_sample=False):
    train_len = int(X.shape[0]*.7)
    tr_X, tr_y = X[:train_len, :], y[:train_len]
    val_X, val_y = X[train_len:], y[train_len:]
    
    if partial_sample is True:
        tr_ix = np.random.choice(tr_X.shape[0], tr_X.shape[0]//3, replace=False)
        val_ix = np.random.choice(val_X.shape[0], val_X.shape[0]//3, replace=False)
        tr_X, tr_y = tr_X[tr_ix,:], tr_y[tr_ix]
        val_X, val_y = val_X[val_ix,:], val_y[val_ix]
        print('Selected hyper-tune data size', tr_X.shape, val_X.shape)
        
    min_score = None
    best_param = None
    for param in list(ParameterGrid(params)):
        use_params = cp(default_params)
        use_params.update(param)
        print('Fitting params:\n', use_params)
        md = FM_FTRL(**use_params)
        md.fit(tr_X, tr_y)
        score = clip_rmse(val_y, md.predict(val_X))
        print(param, score)
        
        if min_score is None or score < min_score:
            best_param = param
            min_score = score
            
    print('Best param:', best_param, '\nscore:', min_score)

In [8]:
fmftrl_default_params = {
    'alpha': .01,
    'beta': .005,
    'L1': 0.001,
    'L2': 0.001,
    'D': train_features.shape[1],
    'D_fm': 300,
    'iters': 10,
    'seed': 719,
    'threads': 4,
    'verbose': 1
}

try_params = {
        'alpha': [.01, .001]
    }
simple_train_val_split_eval(fmftrl_default_params, train_features, train_y, try_params, partial_sample=False)

Fitting params:
 {'alpha': 0.01, 'beta': 0.005, 'L1': 0.001, 'L2': 0.001, 'D': 290474, 'D_fm': 300, 'iters': 10, 'seed': 719, 'threads': 4, 'verbose': 1}
Total e: 163063.3618462964
Total e: 156914.66855343123
Total e: 154858.2807489985
Total e: 153433.38423532326
Total e: 152304.7860509337
Total e: 151350.3796709047
Total e: 150512.01091463843
Total e: 149757.37427453417
Total e: 149066.11821889135
Total e: 148424.23234910058
{'alpha': 0.01} 0.224890068412
Fitting params:
 {'alpha': 0.001, 'beta': 0.005, 'L1': 0.001, 'L2': 0.001, 'D': 290474, 'D_fm': 300, 'iters': 10, 'seed': 719, 'threads': 4, 'verbose': 1}
Total e: 162605.8105925031
Total e: 157812.66076918776
Total e: 155917.88681468592
Total e: 154583.23418639522
Total e: 153523.56667509425
Total e: 152628.44008139416
Total e: 151844.48787003726
Total e: 151139.73385797118
Total e: 150495.25407863565
Total e: 149898.1765089208
{'alpha': 0.001} 0.225598261786
Best param: {'alpha': 0.01} 
score: 0.224890068412


In [9]:
fmftrl_default_params = {
    'alpha': .01,
    'beta': .005,
    'L1': 0.001,
    'L2': 0.001,
    'D': train_features.shape[1],
    'D_fm': 300,
    'iters': 10,
    'seed': 719,
    'threads': 4,
    'verbose': 1
}

try_params = {
        'alpha': [.1]
    }
simple_train_val_split_eval(fmftrl_default_params, train_features, train_y, try_params, partial_sample=False)

Fitting params:
 {'alpha': 0.1, 'beta': 0.005, 'L1': 0.001, 'L2': 0.001, 'D': 290474, 'D_fm': 300, 'iters': 10, 'seed': 719, 'threads': 4, 'verbose': 1}
Total e: 180751.857340916
Total e: 161812.29509048135
Total e: 157461.54321299554
Total e: 154873.23070013468
Total e: 153010.4477423033
Total e: 151543.26509903208
Total e: 150323.42285623215
Total e: 149272.9856554157
Total e: 148345.26674285182
Total e: 147509.73848324403
{'alpha': 0.1} 0.231259183138
Best param: {'alpha': 0.1} 
score: 0.231259183138


In [10]:
fmftrl_default_params = {
    'alpha': .01,
    'beta': .005,
    'L1': 0.001,
    'L2': 0.001,
    'D': train_features.shape[1],
    'D_fm': 300,
    'iters': 10,
    'seed': 719,
    'threads': 4,
    'verbose': 1
}
try_params = {
        'beta': [.005, .001]
    }
simple_train_val_split_eval(fmftrl_default_params, train_features, train_y, try_params, partial_sample=False)

Fitting params:
 {'alpha': 0.01, 'beta': 0.005, 'L1': 0.001, 'L2': 0.001, 'D': 290474, 'D_fm': 300, 'iters': 10, 'seed': 719, 'threads': 4, 'verbose': 1}
Total e: 163063.3618462964
Total e: 156914.66855343123
Total e: 154858.2807489985
Total e: 153433.38423532326
Total e: 152304.7860509337
Total e: 151350.3796709047
Total e: 150512.01091463843
Total e: 149757.37427453417
Total e: 149066.11821889135
Total e: 148424.23234910058
{'beta': 0.005} 0.224890068412
Fitting params:
 {'alpha': 0.01, 'beta': 0.001, 'L1': 0.001, 'L2': 0.001, 'D': 290474, 'D_fm': 300, 'iters': 10, 'seed': 719, 'threads': 4, 'verbose': 1}
Total e: 163101.53399546698
Total e: 156838.80302778535
Total e: 154771.7558730107
Total e: 153343.03419520726
Total e: 152212.25317124857
Total e: 151256.8919258996
Total e: 150418.05760722962
Total e: 149663.3395688035
Total e: 148972.18146632958
Total e: 148330.74288096902
{'beta': 0.001} 0.22489554228
Best param: {'beta': 0.005} 
score: 0.224890068412


In [11]:
fmftrl_default_params = {
    'alpha': .01,
    'beta': .005,
    'L1': 0.001,
    'L2': 0.001,
    'D': train_features.shape[1],
    'D_fm': 300,
    'iters': 10,
    'seed': 719,
    'threads': 4,
    'verbose': 1
}
try_params = {
        'beta': [.0005, .0001]
    }
simple_train_val_split_eval(fmftrl_default_params, train_features, train_y, try_params, partial_sample=False)

Fitting params:
 {'alpha': 0.01, 'beta': 0.0005, 'L1': 0.001, 'L2': 0.001, 'D': 290474, 'D_fm': 300, 'iters': 10, 'seed': 719, 'threads': 4, 'verbose': 1}
Total e: 163110.32268786943
Total e: 156827.42708784735
Total e: 154758.7653412827
Total e: 153329.55609989082
Total e: 152198.49813185897
Total e: 151242.98439171558
Total e: 150404.10617235024
Total e: 149649.39682084578
Total e: 148958.25573596027
Total e: 148316.91094742887
{'beta': 0.0005} 0.224896557243
Fitting params:
 {'alpha': 0.01, 'beta': 0.0001, 'L1': 0.001, 'L2': 0.001, 'D': 290474, 'D_fm': 300, 'iters': 10, 'seed': 719, 'threads': 4, 'verbose': 1}
Total e: 163118.8749386491
Total e: 156817.92773147917
Total e: 154747.92515085213
Total e: 153318.29524776665
Total e: 152187.0293251961
Total e: 151231.36420263376
Total e: 150392.4748867633
Total e: 149637.781688293
Total e: 148946.6567057879
Total e: 148305.41064221735
{'beta': 0.0001} 0.224897380592
Best param: {'beta': 0.0005} 
score: 0.224896557243


In [12]:
fmftrl_default_params = {
    'alpha': .01,
    'beta': .005,
    'L1': 0.001,
    'L2': 0.001,
    'D': train_features.shape[1],
    'D_fm': 300,
    'iters': 10,
    'seed': 719,
    'threads': 4,
    'verbose': 1
}
try_params = {
        'L1': [.01, .001,],
        'L2': [.1, .01, .001,]
    }
simple_train_val_split_eval(fmftrl_default_params, train_features, train_y, try_params, partial_sample=False)

Fitting params:
 {'alpha': 0.01, 'beta': 0.005, 'L1': 0.01, 'L2': 0.1, 'D': 290474, 'D_fm': 300, 'iters': 10, 'seed': 719, 'threads': 4, 'verbose': 1}
Total e: 163034.7479374564
Total e: 157005.05797724854
Total e: 154947.10395502739
Total e: 153519.0532631226
Total e: 152387.47864025593
Total e: 151430.40777908577
Total e: 150589.76109507258
Total e: 149832.98772239292
Total e: 149139.9078763574
Total e: 148496.1843945019
{'L1': 0.01, 'L2': 0.1} 0.224885319254
Fitting params:
 {'alpha': 0.01, 'beta': 0.005, 'L1': 0.01, 'L2': 0.01, 'D': 290474, 'D_fm': 300, 'iters': 10, 'seed': 719, 'threads': 4, 'verbose': 1}
Total e: 163036.70433937426
Total e: 156993.36497784834
Total e: 154933.45061677208
Total e: 153504.4533389069
Total e: 152372.40129069093
Total e: 151415.03517445936
Total e: 150574.232804608
Total e: 149817.36638561243
Total e: 149124.27119566003
Total e: 148480.5261968773
{'L1': 0.01, 'L2': 0.01} 0.224885914704
Fitting params:
 {'alpha': 0.01, 'beta': 0.005, 'L1': 0.01, 'L2': 

In [13]:
fmftrl_default_params = {
    'alpha': .01,
    'beta': .0001,
    'L1': 0.01,
    'L2': 0.1,
    'D': train_features.shape[1],
    'D_fm': 300,
    'iters': 10,
    'seed': 719,
    'threads': 4,
    'verbose': 1
}
try_params = {
        'D_fm': [100, 200, 300, 450, 600]
    }
simple_train_val_split_eval(fmftrl_default_params, train_features, train_y, try_params, partial_sample=False)

Fitting params:
 {'alpha': 0.01, 'beta': 0.0001, 'L1': 0.01, 'L2': 0.1, 'D': 290474, 'D_fm': 100, 'iters': 10, 'seed': 719, 'threads': 4, 'verbose': 1}
Total e: 163080.2898204204
Total e: 158098.66134580094
Total e: 156406.49522797487
Total e: 155236.65013112768
Total e: 154310.2752139485
Total e: 153527.8899524759
Total e: 152841.7921186082
Total e: 152225.26089406933
Total e: 151661.80866681883
Total e: 151140.24800353055
{'D_fm': 100} 0.224856042164
Fitting params:
 {'alpha': 0.01, 'beta': 0.0001, 'L1': 0.01, 'L2': 0.1, 'D': 290474, 'D_fm': 200, 'iters': 10, 'seed': 719, 'threads': 4, 'verbose': 1}
Total e: 163029.8667666383
Total e: 157443.96998070326
Total e: 155538.34910638587
Total e: 154218.02366526844
Total e: 153172.2825135037
Total e: 152288.66449432465
Total e: 151513.26701948917
Total e: 150815.27689637433
Total e: 150176.71307262758
Total e: 149584.32416828081
{'D_fm': 200} 0.224836844616
Fitting params:
 {'alpha': 0.01, 'beta': 0.0001, 'L1': 0.01, 'L2': 0.1, 'D': 290474,

In [14]:
fmftrl_default_params = {
    'alpha': .01,
    'beta': .0001,
    'L1': 0.01,
    'L2': 0.1,
    'D': train_features.shape[1],
    'D_fm': 200,
    'iters': 10,
    'seed': 719,
    'threads': 4,
    'verbose': 1
}
try_params = {
        'iters': [5, 10, 15, 20, 40, 60]
    }
simple_train_val_split_eval(fmftrl_default_params, train_features, train_y, try_params, partial_sample=False)

Fitting params:
 {'alpha': 0.01, 'beta': 0.0001, 'L1': 0.01, 'L2': 0.1, 'D': 290474, 'D_fm': 200, 'iters': 5, 'seed': 719, 'threads': 4, 'verbose': 1}
Total e: 163029.8667666383
Total e: 157443.96998070326
Total e: 155538.34910638587
Total e: 154218.02366526844
Total e: 153172.2825135037
{'iters': 5} 0.225058620947
Fitting params:
 {'alpha': 0.01, 'beta': 0.0001, 'L1': 0.01, 'L2': 0.1, 'D': 290474, 'D_fm': 200, 'iters': 10, 'seed': 719, 'threads': 4, 'verbose': 1}
Total e: 163029.8667666383
Total e: 157443.96998070326
Total e: 155538.34910638587
Total e: 154218.02366526844
Total e: 153172.2825135037
Total e: 152288.66449432465
Total e: 151513.26701948917
Total e: 150815.27689637433
Total e: 150176.71307262758
Total e: 149584.32416828081
{'iters': 10} 0.224836844616
Fitting params:
 {'alpha': 0.01, 'beta': 0.0001, 'L1': 0.01, 'L2': 0.1, 'D': 290474, 'D_fm': 200, 'iters': 15, 'seed': 719, 'threads': 4, 'verbose': 1}
Total e: 163029.8667666383
Total e: 157443.96998070326
Total e: 155538.3

In [1]:
train_len = int(train_features.shape[0]*.7)
tr_X, tr_y = train_features[:train_len, :], train_y[:train_len]
val_X, val_y = train_features[train_len:], train_y[train_len:]

NameError: name 'train_features' is not defined

In [14]:
fmftrl_default_params = {
    'alpha': .01,
    'beta': .0001,
    'L1': 0.001,
    'L2': 0.01,
    'D': train_features.shape[1],
    'D_fm': 100,
    'iters': 10,
    'seed': 719,
    'threads': 4,
    'verbose': 1
}

md = FM_FTRL(**fmftrl_default_params)
md.fit(tr_X, tr_y)

Total e: 164785.8085042756
Total e: 158601.4623680105
Total e: 156963.69157599422
Total e: 155895.15532400517
Total e: 155042.48267711815
Total e: 154363.56260225087
Total e: 153770.30537769498
Total e: 153242.55270082917
Total e: 152748.40941903414
Total e: 152300.87144869703


In [15]:
pd.DataFrame(md.predict(val_X), columns=['fmftrl_pred']).to_csv('fmftrl_cv_charwb24_val_pred.csv', index=False)

In [16]:
del train_features, tr_X, val_X; gc.collect()

6

In [25]:
all_features = load_npz('all_features_cv_charwb24.npz')
train_len = train_y.shape[0]
test_features = all_features[train_len:, :]
del all_features; gc.collect()

278

In [26]:
test_features.shape[0]

508438

In [27]:
test_pred = md.predict(test_features)

In [28]:
pd.DataFrame(test_pred, columns=['fmftrl_pred']).to_csv('fmftrl_cv_charwb24_test_pred.csv', index=False)

In [29]:
test_df = pd.read_csv("data/test.csv", usecols=['item_id'])

In [31]:
pd.DataFrame(np.clip(test_pred,0,1), 
             index=test_df.item_id,
             columns=['deal_probability']).to_csv('fmftrl_cv_charwb24_submission.csv')